In [1]:
import os
import glob
import time
import json
from tqdm.notebook import tqdm
import cv2
import numpy as np
from ekmipy import ekmi

In [2]:
order = 20
p = 0.5
N = 64

so_path = './ekmi.so'
feature_single_dir = 'extracted/feature_{}_{}'.format(N, order)
feature_path = 'extracted/ekmi_{}_{}.npy'.format(N, order)
fnames_path = 'extracted/fnames.json'
get_ekmi = ekmi(so_path)

if not os.path.isdir(feature_single_dir):
    os.mkdir(feature_single_dir)

In [3]:
data_dir = '/Users/kx/Docs/github/git_cccr_sl/chinese-calligraphy-recognition-sl/data/'
# fnames = glob.glob(data_dir + '*/clean/*/*.png')
# fnames = sorted(fnames, key=lambda x: x.split('/')[-2])

with open(fnames_path, 'r') as f:
    fnames_short = json.load(f)
    
fnames = [data_dir + x for x in fnames_short]

In [4]:
feature = np.zeros((len(fnames), order, order))

for i, fname in tqdm(enumerate(fnames), total=len(fnames)):
    
    path = os.path.join(feature_single_dir, str(i) + '.npy')
    
    if not os.path.isfile(path):
        img = cv2.imread(fname)
        img = cv2.resize(img, (N, N))
        img = (img[:, :, 0] > 0).astype('int32')
        feature[i, :, :] = get_ekmi(order, img, p)


        with open(path, 'wb') as f:
            np.save(f, feature[i, :, :])
            
    else:
        with open(path, 'rb') as f:
            feature[i, :, :] = np.load(f)

  0%|          | 0/10035 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [55]:
with open(feature_path, 'wb') as f:
    np.save(f, feature)

with open(feature_path, 'rb') as f:
    feature_loaded = np.load(f)